$\vec{y} = \begin{pmatrix} x \\ y \\ z \end{pmatrix} = f(\vec{p}) = f \left( \begin{pmatrix} \phi \\ \theta \end{pmatrix} \right) = \begin{pmatrix} a \sin \phi \cos \theta \\ b \sin \phi \sin \theta \\ c \cos \phi \end{pmatrix}$

<img src='http://www4c.wolframalpha.com/Calculate/MSP/MSP22911ciacb27a962182000006a956d4787hc1b3i?MSPStoreType=image/gif&s=1&w=151.&h=72.' />
<img src='http://www4c.wolframalpha.com/Calculate/MSP/MSP22841ciacb27a962182000000d31bi4d51bd7524?MSPStoreType=image/gif&s=1&w=337.&h=90.' />
<img src='http://www4c.wolframalpha.com/Calculate/MSP/MSP28221hc4fbh4e87bf6i1000026a9ghg3g32ba4bg?MSPStoreType=image/gif&s=50&w=449.&h=140.' />

$􏰡dN \begin{pmatrix}x\\y\end{pmatrix} = \begin{pmatrix} 􏰠a & b \\ c & d \end{pmatrix} \begin{pmatrix}x\\y\end{pmatrix}$

$\begin{pmatrix} 􏰠a & b \\ c & d \end{pmatrix} = \frac{1}{EG-F^2} \begin{pmatrix} MF-LG & NF-MG \\ LF-ME & MF-NE \end{pmatrix}$

$u = \theta \\v = \phi$

In [74]:
#%matplotlib inline
#%config InlineBackend.figure_format = 'svg'

from __future__ import division

import numpy as np

from util import plotutil

from infotopo import predict


sin, cos, sqrt, pi = np.sin, np.cos, np.sqrt, np.pi

def get_S(a, b, c):
    E = lambda u, v: sin(v)**2 * (a**2 * sin(u)**2 + b**2 * cos(u)**2)
    F = lambda u, v: (b**2 - a**2) * sin(u) * cos(u) * sin(v) * cos(v)
    G = lambda u, v: cos(v)**2 * (a**2 * cos(u)**2 + b**2 * sin(u)**2) + c**2 * sin(v)**2
    D = lambda u, v: sqrt(a**2 * b**2 * cos(v)**2 + a**2 * c**2 * sin(u)**2 * sin(v)**2 + b**2 * c**2 * cos(u)**2 * sin(v)**2)
    L = lambda u, v: a * b * c * sin(v)**2 / D(u,v)
    M = lambda u, v: 0
    N = lambda u, v: a * b * c / D(u,v)

    gdet = lambda u, v: E(u,v) * G(u,v) - F(u,v)**2  # determinant of g
    dN11 = lambda u, v: (M(u,v) * F(u,v) - L(u,v) * G(u,v)) / gdet(u,v)
    dN12 = lambda u, v: (N(u,v) * F(u,v) - M(u,v) * G(u,v)) / gdet(u,v)
    dN21 = lambda u, v: (L(u,v) * F(u,v) - M(u,v) * E(u,v)) / gdet(u,v)
    dN22 = lambda u, v: (M(u,v) * F(u,v) - N(u,v) * E(u,v)) / gdet(u,v)

    dN0 = lambda u, v: np.array([[dN11(u,v), dN12(u,v)], [dN21(u,v), dN22(u,v)]])

    S = lambda ph, th: -dN0(th, ph)
    return S

In [75]:
def get_predict(a, b, c):
    def f(p):
        ph, th = p
        return np.array([a * sin(ph) * cos(th),
                         b * sin(ph) * sin(th),
                         c * cos(ph)])
    def Df(p):
        ph, th = p
        return np.array([[a * cos(ph) * cos(th), -a * sin(ph) * sin(th)],
                         [b * cos(ph) * sin(th), b * sin(ph) * cos(th)],
                         [-c * sin(ph), 0]])
    pred = predict.Predict(f=f, Df=Df, p0=[pi/2, 0], pids=['phi','theta'], yids=['x','y','z'])
    return pred

In [76]:
a, b, c = 2, 1, 1

S = get_S(a, b, c)
pred = get_predict(a, b, c)

pred.plot_image(np.linspace(0, pi, 31), np.linspace(0, 2*pi, 31), alpha=0.05, figsize=(8,8), 
                xyzlims=[[-2,2]]*3, xyzlabels=pred.yids,
                pts=[pred(), pred((0,0)), pred((pi/2,0)), pred((pi,0))], 
                cs=['k', 'b', 'g', 'r'])


In [78]:
A = S(pi/4, pi/4)
lams, vs = np.linalg.eig(A)
print A
print lams
print vs

[[ 0.59736944  0.51203095]
 [ 0.25601548  0.85338492]]
[ 0.34135397  1.10940039]
[[-0.89442719 -0.70710678]
 [ 0.4472136  -0.70710678]]


In [80]:
np.dot(vs[0], vs[1])

0.099999999999999922

For an ellipse parameterized by $\langle a \cos t, b \sin t \rangle$, its curvature is:

$\kappa(t) = ||\frac{d \hat{T}(t)/dt}{v}|| = \frac{ab}{\left(a^2 \sin^2{t} + b^2 \cos^2 t\right)^{\frac{3}{2}}}$

See [here](http://math.stackexchange.com/questions/527538/how-to-calculate-the-curvature-of-an-ellipse) for a reference. 

In [64]:
ts = np.linspace(0, 2*pi, 36)
xys = np.array([(a*cos(t), b*sin(t)) for t in ts])

plotutil.scatterplot(*xys.T, c=ts, cmap='jet', xylims=[[-2.5,2.5]]*2, s=30, figsize=(4,4), edgecolor='none')

Verify the calculations: 

Here is a [reference](https://en.wikipedia.org/wiki/Radius_of_curvature_%28mathematics%29#Ellipses). 

In [63]:
def get_ellipse_curvature(a, b, t):
    return a * b / np.power(a**2 * sin(t)**2 + b**2 * cos(t)**2, 3/2)

print get_ellipse_curvature(2,1,0)
print get_ellipse_curvature(2,1,pi/2)

2.0
0.25
